## Grouping by VisitNumber 
- TripType, Weekday 변수는 VisitNumber별 최빈값을 넣음
- DepartmentDescription, FinelineNumber 변수는 더미변수로 만들고, VisitNumber별 해당 카테고리에 대한 갯수를 넣음  
- Upc, ScanCount 변수 제외 

In [178]:
mytrain = pd.read_csv("data/mytrain.csv")
len(mytrain)

647052

In [179]:
test = pd.read_csv("data/test.csv")
len(test)

653646

In [196]:
len(mytrain["VisitNumber"].unique())

95674

In [182]:
len(test["VisitNumber"].unique())

95674

In [253]:
submission = pd.read_csv("data/sample_submission.csv")
submission.head()

,VisitNumber,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [184]:
len(submission)

95674

In [3]:
# 최빈값 함수
from collections import Counter
def mode(x):
    counts = Counter(x)
    max_count = max(counts.values())
    ls = [x_i for x_i, count in counts.items() if count == max_count]
    return ls[0]

In [27]:
# value를 개수로...
def index_VisitNumber(data):
    # VisitNumber별 TripType, Weekday grouping(최빈값) 후 merge
    df_TT = data.groupby('VisitNumber').agg(mode)['TripType'].reset_index()
    df_Weekday = data.groupby('VisitNumber').agg(mode)['Weekday'].reset_index()
    df_merge1 = pd.merge(df_TT, df_Weekday)
    
    # Weekday 숫자로 변형
    wd = {
        "Monday": 0,
        "Tuesday": 1,
        "Wednesday": 2,
        "Thursday": 3, 
        "Friday": 4,
        "Saturday": 5,
        "Sunday": 6,
    }
    df_merge1["Weekday"] = df_merge1["Weekday"].apply(lambda x: wd[x])

    
    # VisitNumber별 DepartmentDescription, FinelineNumber 더미변수로 grouping 후 merge
    df_DD = data.pivot_table(values='Count', index=['VisitNumber'], columns=['DepartmentDescription'], aggfunc=np.sum, fill_value=0)
    df_DD = df_DD.reset_index()
    df_FN = data.pivot_table(values='Count', index=['VisitNumber'], columns=['FinelineNumber'], aggfunc=np.sum, fill_value=0)
    df_FN = df_FN.reset_index()
    df_merge2 = pd.merge(df_DD, df_FN)
    train_IndexVN = pd.merge(df_merge1, df_merge2)
    train_IndexVN = train_IndexVN.set_index('VisitNumber')
    
    return train_IndexVN

In [ ]:
Index_VN = index_VisitNumber(train)

In [ ]:
Index_VN.to_csv("data/train_indexVN.csv")

In [219]:
df = mytrain.groupby('VisitNumber').agg(mode)['DepartmentDescription'].reset_index()
df2 = pd.get_dummies(df.DepartmentDescription).astype(int, copy=False)
len(df2)

95674

In [211]:
len(df["DepartmentDescription"].unique())

68

In [210]:
len(df)

95674

In [220]:
# value를 ScanCount의 합으로...
def index_VisitNumber2(data):
    # VisitNumber별 TripType, Weekday grouping(최빈값) 후 merge
    df_TT = data.groupby('VisitNumber').agg(mode)['TripType'].reset_index()
    df_Weekday = data.groupby('VisitNumber').agg(mode)['Weekday'].reset_index()
    df_merge1 = pd.merge(df_TT, df_Weekday)
    
    # Weekday 숫자로 변형
    wd = {
        "Monday": 0,
        "Tuesday": 1,
        "Wednesday": 2,
        "Thursday": 3, 
        "Friday": 4,
        "Saturday": 5,
        "Sunday": 6,
    }
    df_merge1["Weekday"] = df_merge1["Weekday"].apply(lambda x: wd[x])

    
    df_DD = data.groupby('VisitNumber').agg(mode)['DepartmentDescription'].reset_index()
    df_DD = pd.get_dummies(df_DD.DepartmentDescription).astype(int, copy=False)
    train_IndexVN = pd.concat([df_merge1, df_DD], axis=1)
    
    # VisitNumber별 DepartmentDescription, FinelineNumber 더미변수로 grouping 후 merge
    #df_DD = data.pivot_table(values='ScanCount', index=['VisitNumber'], columns=['DepartmentDescription'], aggfunc=np.sum, fill_value=0, dropna=False)
    #df_DD = df_DD.reset_index()
    #df_FN = data.pivot_table(values='ScanCount', index=['VisitNumber'], columns=['FinelineNumber'], aggfunc=np.sum, fill_value=0)
    #df_FN = df_FN.reset_index()
    #df_merge2 = pd.merge(df_DD, df_FN)
    #train_IndexVN = pd.merge(df_merge1, df_merge2)
    #train_IndexVN = pd.merge(df_merge1, df_DD)
    #train_IndexVN = train_IndexVN.set_index('VisitNumber')
    
    return train_IndexVN

In [221]:
# test data 가공
def index_VisitNumber2_test(data):
    # VisitNumber별 TripType, Weekday grouping(최빈값) 후 merge
    df_Weekday = data.groupby('VisitNumber').agg(mode)['Weekday'].reset_index()
    
    # Weekday 숫자로 변형
    wd = {
        "Monday": 0,
        "Tuesday": 1,
        "Wednesday": 2,
        "Thursday": 3, 
        "Friday": 4,
        "Saturday": 5,
        "Sunday": 6,
    }
    df_Weekday["Weekday"] = df_Weekday["Weekday"].apply(lambda x: wd[x])

    
    df_DD = data.groupby('VisitNumber').agg(mode)['DepartmentDescription'].reset_index()
    df_DD = pd.get_dummies(df_DD.DepartmentDescription).astype(int, copy=False)
    test_IndexVN = pd.concat([df_Weekday, df_DD], axis=1)
    
    # VisitNumber별 DepartmentDescription, FinelineNumber 더미변수로 grouping 후 merge
    #df_DD = data.pivot_table(values='ScanCount', index=['VisitNumber'], columns=['DepartmentDescription'], aggfunc=np.sum, fill_value=0, dropna=False)
    #df_DD = df_DD.reset_index()
    #df_FN = data.pivot_table(values='ScanCount', index=['VisitNumber'], columns=['FinelineNumber'], aggfunc=np.sum, fill_value=0)
    #df_FN = df_FN.reset_index()
    #df_merge2 = pd.merge(df_DD, df_FN)
    #test_IndexVN = pd.merge(df_Weekday, df_DD)
    #test_IndexVN = train_IndexVN.set_index('VisitNumber')
    
    return test_IndexVN

In [222]:
mytrain1 = index_VisitNumber2(mytrain).astype(int)
mytrain1.head()

,VisitNumber,TripType,Weekday,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
0,5,999,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,30,4,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,8,26,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,8,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10,8,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [223]:
len(mytrain1)

95674

In [224]:
mytest1 = index_VisitNumber2_test(test)
mytest1.head()

,VisitNumber,Weekday,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
0,1,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [225]:
len(mytest1)

95674

In [226]:
a = mytrain1.columns.unique()
b = mytest1.columns.unique()
set(a) - set(b)

{'TripType'}

In [227]:
len(a), len(b)

(70, 69)

### Random Forest

In [228]:
from sklearn.metrics import *
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
import sklearn

In [229]:
mytrain1_col = mytrain1.drop(['TripType'], axis=1)
dfX = mytrain1_col
dfy = mytrain1.TripType

In [242]:
forest = RandomForestClassifier(n_estimators = 100, max_features=2)

In [243]:
forest = forest.fit(dfX, dfy)

In [244]:
prediction = forest.predict(mytest1)

In [245]:
len(prediction)

95674

In [249]:
trial = pd.DataFrame(columns=["VisitNumber", "TripType"])
trial["VisitNumber"] = test["VisitNumber"]
trial["TripType"] = prediction
dum_TT = pd.get_dummies(trial.TripType).astype(int, cbopy=False)

In [252]:
dum_TT.to_csv("data/trial_2.csv")

##### train_test_split

In [230]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(dfX, dfy, test_size=0.25, random_state=0)

In [143]:
print("Train Score: {:.3f}".format(forest.score(X_train, y_train)))
print("Test Score: {:.3f}".format(forest.score(X_test, y_test)))

Train Score: 0.998
Test Score: 0.606


In [144]:
confusion_matrix(y_train, forest.predict(X_train))

array([[2718,    0,    0, ...,    0,    0,    3],
       [   0,  257,    3, ...,    0,    0,    0],
       [   0,    1, 3384, ...,    0,    0,    0],
       ...,
       [   0,    0,    0, ...,  638,    0,    0],
       [   0,    0,    0, ...,    0,  880,    0],
       [  23,    0,    1, ...,    0,    0, 6301]])

In [145]:
confusion_matrix(y_test, forest.predict(X_test))

array([[ 722,    0,    1, ...,    0,    0,  138],
       [   0,    9,   62, ...,    0,    0,    0],
       [   2,   49,  909, ...,    0,    1,    9],
       ...,
       [   4,    0,   12, ...,    5,    0,    1],
       [   0,    0,    3, ...,    1,    4,    0],
       [ 136,    1,   24, ...,    0,    0, 1469]])

In [147]:
print(classification_report(y_test, forest.predict(X_test)))

             precision    recall  f1-score   support

          3       0.74      0.78      0.76       921
          4       0.13      0.10      0.12        86
          5       0.69      0.75      0.72      1208
          6       0.61      0.65      0.63       306
          7       0.60      0.67      0.63      1400
          8       0.66      0.77      0.71      3052
          9       0.58      0.70      0.64      2427
         12       0.25      0.06      0.10        82
         15       0.52      0.33      0.40       256
         18       0.32      0.25      0.28       145
         19       0.34      0.26      0.30        95
         20       0.53      0.53      0.53       166
         21       0.54      0.51      0.53       152
         22       0.39      0.27      0.32       224
         23       0.44      0.29      0.35        38
         24       0.55      0.53      0.54       630
         25       0.65      0.67      0.66       950
         26       0.45      0.27      0.34   